# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
! pip install scikit-learn --upgrade

    100% |████████████████████████████████| 7.1MB 4.9MB/s eta 0:00:01  1% |▍                               | 92kB 2.9MB/s eta 0:00:03    4% |█▋                              | 348kB 4.7MB/s eta 0:00:02    75% |████████████████████████▏       | 5.3MB 20.7MB/s eta 0:00:01    95% |██████████████████████████████▌ | 6.7MB 27.6MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.19.1
    Uninstalling scikit-learn-0.19.1:
      Successfully uninstalled scikit-learn-0.19.1


In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)
df.dropna(subset=['related'],inplace=True) # I found that there are messages with null responses, so i will drop those rows
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'], axis=1)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

In [5]:
print(X[1])
print(tokenize(X[1]))
print(Y[:1])

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over']
   related  request  offer  aid_related  medical_help  medical_products  \
0      1.0      0.0    0.0          0.0           0.0               0.0   

   search_and_rescue  security  military  water      ...        aid_centers  \
0                0.0       0.0       0.0    0.0      ...                0.0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                   0.0              0.0     0.0    0.0   0.0         0.0   

   cold  other_weather  direct_report  
0   0.0            0.0            0.0  

[1 rows x 35 columns]


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 500)
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight=None,
                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], Y_pred[:, i]))

related
              precision    recall  f1-score   support

         0.0       0.40      0.07      0.11      1555
         1.0       0.76      0.97      0.85      4915
         2.0       0.00      0.00      0.00        40

    accuracy                           0.75      6510
   macro avg       0.39      0.35      0.32      6510
weighted avg       0.67      0.75      0.67      6510

request
              precision    recall  f1-score   support

         0.0       0.84      0.97      0.90      5388
         1.0       0.45      0.12      0.19      1122

    accuracy                           0.82      6510
   macro avg       0.65      0.54      0.54      6510
weighted avg       0.77      0.82      0.78      6510

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6483
         1.0       0.00      0.00      0.00        27

    accuracy                           1.00      6510
   macro avg       0.50      0.50      0.50      651

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6298
         1.0       0.00      0.00      0.00       212

    accuracy                           0.97      6510
   macro avg       0.48      0.50      0.49      6510
weighted avg       0.94      0.97      0.95      6510

death
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6195
         1.0       0.00      0.00      0.00       315

    accuracy                           0.95      6510
   macro avg       0.48      0.50      0.49      6510
weighted avg       0.91      0.95      0.93      6510

other_aid
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93      5683
         1.0       0.00      0.00      0.00       827

    accuracy                           0.87      6510
   macro avg       0.44      0.50      0.47      6510
weighted avg       0.76      0.87      0.81      6510

inf

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x7f45cc30fae8>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                      dual=False,
                                                      fit_intercept=True,
                                                      intercept_scaling=1,
                                                      l1_ratio=None, max_it

In [10]:
parameters = {'clf__estimator__C': [0.5, 1, 1.5],
              'tfidf__smooth_idf': (True,False),
              'clf__estimator__penalty': ['l1', 'l2', 'elasticnet']}

cv = GridSearchCV(pipeline, param_grid=parameters,cv=2, verbose=1, n_jobs=-1)
cv.fit(X_train, y_train)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_se

GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

###### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
Y_pred = cv.predict(X_test)
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], Y_pred[:, i]))

related
              precision    recall  f1-score   support

         0.0       0.44      0.04      0.08      1555
         1.0       0.76      0.98      0.86      4915
         2.0       0.00      0.00      0.00        40

    accuracy                           0.75      6510
   macro avg       0.40      0.34      0.31      6510
weighted avg       0.68      0.75      0.67      6510

request
              precision    recall  f1-score   support

         0.0       0.84      0.98      0.90      5388
         1.0       0.47      0.10      0.16      1122

    accuracy                           0.83      6510
   macro avg       0.65      0.54      0.53      6510
weighted avg       0.77      0.83      0.77      6510

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6483
         1.0       0.00      0.00      0.00        27

    accuracy                           1.00      6510
   macro avg       0.50      0.50      0.50      651

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6298
         1.0       0.00      0.00      0.00       212

    accuracy                           0.97      6510
   macro avg       0.48      0.50      0.49      6510
weighted avg       0.94      0.97      0.95      6510

death
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6195
         1.0       0.00      0.00      0.00       315

    accuracy                           0.95      6510
   macro avg       0.48      0.50      0.49      6510
weighted avg       0.91      0.95      0.93      6510

other_aid
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93      5683
         1.0       0.00      0.00      0.00       827

    accuracy                           0.87      6510
   macro avg       0.44      0.50      0.47      6510
weighted avg       0.76      0.87      0.81      6510

inf

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
X = df[['message','genre']]

In [13]:
#Let's try enhancing our model by adding the genre column after transforming it with the One Hot Encoder
# At first, we use a logistic regression classifier. now we are going to use more complex model which is the Random forest

Text_pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                          ('tfidf', TfidfTransformer())])


Column_Transformer = ColumnTransformer(
            transformers=[('Categorical_Transform', OneHotEncoder(), ['genre']),
                          ('Text_Transform',Text_pipeline,'message')
                         ])

New_pipeline = Pipeline([
        ('ColumnTransformer', Column_Transformer ),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [14]:
X.columns

Index(['message', 'genre'], dtype='object')

In [15]:
Column_Transformer.fit_transform(X)

<26039x37359 sparse matrix of type '<class 'numpy.float64'>'
	with 619300 stored elements in Compressed Sparse Row format>

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 500)
New_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ColumnTransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('Categorical_Transform',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['genre']),
                                                 ('Text_Transform',
                                                  Pipeline(memory=None,
                                                           s

In [19]:
New_pipeline.get_params()

{'memory': None,
 'steps': [('ColumnTransformer',
   ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                     transformer_weights=None,
                     transformers=[('Categorical_Transform',
                                    OneHotEncoder(categories='auto', drop=None,
                                                  dtype=<class 'numpy.float64'>,
                                                  handle_unknown='error',
                                                  sparse=True),
                                    ['genre']),
                                   ('Text_Transform',
                                    Pipeline(memory=None,
                                             steps=[('vect',
                                                     CountVectorizer(analyzer='word',
                                                                     binary=False,
                                                                     d...
     

In [23]:
Y_pred = New_pipeline.predict(X_test)
for i, col in enumerate(y_test):
    print(col)
    print(classification_report(y_test[col], Y_pred[:, i]))

related
              precision    recall  f1-score   support

         0.0       0.38      0.02      0.04      1555
         1.0       0.76      0.99      0.86      4915
         2.0       0.50      0.03      0.05        40

    accuracy                           0.75      6510
   macro avg       0.55      0.34      0.31      6510
weighted avg       0.67      0.75      0.66      6510

request
              precision    recall  f1-score   support

         0.0       0.83      0.99      0.91      5388
         1.0       0.56      0.03      0.06      1122

    accuracy                           0.83      6510
   macro avg       0.70      0.51      0.48      6510
weighted avg       0.79      0.83      0.76      6510

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6483
         1.0       0.00      0.00      0.00        27

    accuracy                           1.00      6510
   macro avg       0.50      0.50      0.50      651

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6298
         1.0       1.00      0.00      0.01       212

    accuracy                           0.97      6510
   macro avg       0.98      0.50      0.50      6510
weighted avg       0.97      0.97      0.95      6510

death
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6195
         1.0       0.00      0.00      0.00       315

    accuracy                           0.95      6510
   macro avg       0.48      0.50      0.49      6510
weighted avg       0.91      0.95      0.93      6510

other_aid
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93      5683
         1.0       0.15      0.00      0.01       827

    accuracy                           0.87      6510
   macro avg       0.51      0.50      0.47      6510
weighted avg       0.78      0.87      0.81      6510

inf

### 9. Export your model as a pickle file

In [18]:
import pickle
pickle.dump(New_pipeline, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.